Event loop: a first class object responsible for efficiently handling I/O events, system events, and application context changes.

Future: a data structure representing the result of work that has not been completed yet.

The event loop can watch for a Future object to be set to done, allowing one part of an application to wait for another part to finish some work. Besides futures, asyncio includes other concurrency primitives such as locks and semaphores.

A `Task` is a subclass of `Future` that knows how to wrap and manage the execution for a coroutine. Tasks can be scheduled with the event loop to run when the resources they need are available, and to produce a result that can be consumed by other coroutines.

In [1]:
import asyncio

async def coroutine():
    print('in coroutine')
    return 'result'
    
event_loop = asyncio.get_event_loop()
try:
    print('starting coroutine')
    coro = coroutine()
    print('entering event loop')
    return_value = event_loop.run_until_complete(coro)
    print('Got:', return_value)
finally:
    print('closing eventloop')
    #event_loop.close()

starting coroutine
entering event loop
in coroutine
Got: result
closing eventloop


In [12]:
async def outer():
    print('in outer')
    print('waiting for result1')
    result1 = await phase1()
    print('waiting for result2')
    result2 = await phase2(result1)
    return result1, result2
    
async def phase1():
    print('in phase1')
    return 'result1'

async def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)

event_loop = asyncio.get_event_loop()

try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    pass

in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')


`call_soon()` can be used to schedule the call for the next iteration of the loop. Any extra positional arguments after the function are passed to the callback when it is invoked. 

In [11]:
import functools

def callback(arg, *, kwarg='default'):
    print('callback ivoked with {} and {}'.format(arg, kwarg))
    
async def main(loop):
    print('registering callback')
    loop.call_soon(callback, 1)
    wrapped = functools.partial(callback, kwarg='not default')
    loop.call_soon(wrapped, 2)
    
    await asyncio.sleep(0.1)
    
event_loop = asyncio.get_event_loop()

try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    pass

entering event loop
registering callback
callback ivoked with 1 and default
callback ivoked with 2 and not default


In [18]:
def callback(n):
    print('callback {} invoked'.format(n))
    
async def main(loop):
    print('registering callbacks')
    loop.call_later(2.2, callback, 1)
    loop.call_later(2.2, callback, 2)
    loop.call_soon(callback, 3)
    
    await asyncio.sleep(5.4)
    
event_loop = asyncio.get_event_loop()

try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    pass

entering event loop
registering callbacks
callback 3 invoked
callback 1 invoked
callback 2 invoked


In [20]:
import time

def callback(n, loop):
    print('callback {} invoked at {}'.format(n, loop.time()))
    
async def main(loop):
    now = loop.time()
    print('clock time: {}'.format(time.time()))
    print('loop time: {}'.format(now))
    
    print('registering callbacks')
    loop.call_at(now + 0.2, callback, 1, loop)
    loop.call_at(now + 0.1, callback, 2, loop)
    loop.call_soon(callback, 3, loop)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()

try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    pass

entering event loop
clock time: 1484555223.475652
loop time: 368431.281
registering callbacks
callback 3 invoked at 368431.281
callback 2 invoked at 368431.375
callback 1 invoked at 368431.484
